
# Perceptual Image Generator — **Calibrated Display Mapping**
This notebook generates Gaussian-blob stimuli in additive white noise (as in the paper) **and**
uses a *fixed, linear* mapping from cd/m² → 8‑bit so the images look like the article:
- Mean luminance ≈ **28 cd/m²** maps to **mid‑gray (128)**.
- No per-image contrast stretching (preserves relative SNR).
- All **computations** (mean, SD, RMS, SNR) are done on **float cd/m²** arrays.

**Sections**
1. Config & constants
2. Stimulus generation (float)
3. Calibrated mapping for display/save
4. Visual demos (absent vs present, components)
5. Batch export to dataset (PNG + optional .npy)
6. Verification (single‑trial sanity + dataset checks)


In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from PIL import Image
import csv, json, math, random

# ====== Experiment constants ======
MEAN_LUMINANCE = 28.0          # cd/m^2
NOISE_SD = 4.375               # cd/m^2 (RMS)
FIELD_DEG = 15.0               # degrees
PIXELS_PER_DEG = 1.0 / 0.037   # ≈27.027 px/deg
IMG_SIZE = int(round(FIELD_DEG * PIXELS_PER_DEG))  # ≈405 px
SIGMA_DEG = 0.5
SIGMA_PX  = SIGMA_DEG * PIXELS_PER_DEG             # ≈13.5 px

# Peak contrasts → amplitudes = contrast * MEAN_LUMINANCE
PEAK_CONTRAST_EQUAL  = 0.02
PEAK_CONTRAST_WEAK   = 0.005
PEAK_CONTRAST_STRONG = 0.09

TARGET_SNRS = {"equal": 3.07, "weak": 0.77, "strong": 13.8}

# ====== Display calibration (fixed linear mapping, like the article) ======
DISPLAY_MEAN_8BIT = 128.0                   # where 28 cd/m^2 should land on 0–255
DISPLAY_GAIN      = DISPLAY_MEAN_8BIT / MEAN_LUMINANCE   # ≈ 128/28 ≈ 4.57

# RNG (set to an int for reproducibility)
RANDOM_SEED = None
if RANDOM_SEED is not None:
    np.random.seed(RANDOM_SEED)

IMG_SIZE, SIGMA_PX, DISPLAY_GAIN


In [ ]:

def gaussian_blob(size, sigma_px, amplitude, center=None):
    """
    2D Gaussian (signal-only) in luminance units (cd/m^2).
    amplitude is the peak increment above baseline.
    """
    h = w = size
    if center is None:
        cy, cx = (h-1)/2.0, (w-1)/2.0
    else:
        cy, cx = center
    y = np.arange(h)[:, None]
    x = np.arange(w)[None, :]
    g = np.exp(-(((y - cy)**2 + (x - cx)**2) / (2*sigma_px**2)))
    return amplitude * g

def white_noise(size, mean_lum=MEAN_LUMINANCE, sigma=NOISE_SD):
    """Gaussian white noise field in cd/m^2."""
    return np.random.normal(mean_lum, sigma, (size, size))

def make_stimulus(present=True, peak_contrast=PEAK_CONTRAST_EQUAL):
    """
    Returns float cd/m^2 arrays: combined, noise, signal.
    """
    noise = white_noise(IMG_SIZE)
    if present:
        amp = peak_contrast * MEAN_LUMINANCE
        signal = gaussian_blob(IMG_SIZE, SIGMA_PX, amp)
        combined = noise + signal
    else:
        signal = np.zeros((IMG_SIZE, IMG_SIZE), dtype=float)
        combined = noise
    return combined, noise, signal

def to_uint8_visible(lum_float):
    """
    Calibrated mapping: cd/m^2 → 8-bit counts with fixed linear gain.
    Ensures ~28 cd/m^2 maps to ~128 mid-gray; preserves relative signal/noise.
    """
    counts = lum_float * DISPLAY_GAIN
    return np.clip(counts, 0, 255).astype(np.uint8)

def to_uint8_visible_signal(signal_float):
    """
    For visualizing/saving the signal-only image, add mean luminance before mapping
    so mid-gray is preserved and the blob is visible.
    """
    return to_uint8_visible(MEAN_LUMINANCE + signal_float)


## Demo: Absent vs Present (Equal condition, calibrated mapping)

In [ ]:

fig, axes = plt.subplots(1, 2, figsize=(8,4))

img_abs, noise_abs, sig_abs = make_stimulus(present=False, peak_contrast=PEAK_CONTRAST_EQUAL)
img_pre, noise_pre, sig_pre = make_stimulus(present=True,  peak_contrast=PEAK_CONTRAST_EQUAL)

axes[0].imshow(to_uint8_visible(img_abs), cmap='gray', vmin=0, vmax=255)
axes[0].set_title('Absent (calibrated)'); axes[0].axis('off')

axes[1].imshow(to_uint8_visible(img_pre), cmap='gray', vmin=0, vmax=255)
axes[1].set_title('Present (calibrated)'); axes[1].axis('off')

plt.tight_layout(); plt.show()

print(f"DISPLAY_GAIN = {DISPLAY_GAIN:.4f} (28 cd/m^2 → {DISPLAY_MEAN_8BIT} counts).")


## Component view: Noise-only, Signal-only, Combined (Equal, calibrated)

In [ ]:

img, noise, signal = make_stimulus(present=True, peak_contrast=PEAK_CONTRAST_EQUAL)

fig, axes = plt.subplots(1, 3, figsize=(12,4))
axes[0].imshow(to_uint8_visible(noise), cmap='gray', vmin=0, vmax=255)
axes[0].set_title('Noise-only (calibrated)'); axes[0].axis('off')

axes[1].imshow(to_uint8_visible_signal(signal), cmap='gray', vmin=0, vmax=255)
axes[1].set_title('Signal-only (around mean)'); axes[1].axis('off')

axes[2].imshow(to_uint8_visible(img), cmap='gray', vmin=0, vmax=255)
axes[2].set_title('Combined (present, calibrated)'); axes[2].axis('off')
plt.tight_layout(); plt.show()


## Verification: numeric checks on float arrays (cd/m²)

In [ ]:

def signal_energy(blob): 
    return float(np.sum(blob**2))

def empirical_snr(blob, noise_sigma=NOISE_SD): 
    return np.sqrt(signal_energy(blob)) / noise_sigma

def check_condition(label, peak_contrast, target_snr):
    stim, noise, signal = make_stimulus(present=True, peak_contrast=peak_contrast)
    mean_lum = float(np.mean(noise))
    sd_noise = float(np.std(noise, ddof=1))
    rms = sd_noise / mean_lum
    amp = peak_contrast * MEAN_LUMINANCE
    snr = empirical_snr(signal, NOISE_SD)

    print(f"--- {label} condition ---")
    print(f"Image size          = {stim.shape} (target ~405×405)")
    print(f"Mean luminance      = {mean_lum:.2f}   (target ~28)")
    print(f"Noise SD            = {sd_noise:.3f}   (target ~4.375)")
    print(f"RMS contrast        = {rms:.4f}   (target ~0.1562)")
    print(f"Signal σ (px)       = {SIGMA_PX:.2f}   (target ~13.5)")
    print(f"Signal peak ampl.   = {amp:.3f}   cd/m²")
    print(f"Empirical SNR       = {snr:.2f}   (target ~{target_snr})")

# Run checks
check_condition("Equal (2%)",   PEAK_CONTRAST_EQUAL,  TARGET_SNRS["equal"])
check_condition("Weak (0.5%)",  PEAK_CONTRAST_WEAK,   TARGET_SNRS["weak"])
check_condition("Strong (9%)",  PEAK_CONTRAST_STRONG, TARGET_SNRS["strong"])


## Export dataset (calibrated PNGs for LLMs, optional .npy floats)

In [ ]:

# === CONFIG ===
OUTPUT_DIR = Path("perceptual_dataset_calibrated")  # change if desired
TRIALS_PER_CONDITION = 200
SAVE_NPY = True         # save float arrays for verification/analysis
SAVE_SIGNAL_PNG = True  # save signal-only PNGs (visual QA)

def ensure_dirs(base: Path, cond: str):
    for label in ("absent", "present"):
        (base / cond / label / "combined").mkdir(parents=True, exist_ok=True)
        (base / cond / label / "noise").mkdir(parents=True, exist_ok=True)
        (base / cond / label / "signal").mkdir(parents=True, exist_ok=True)

def save_trial(out_dir: Path, cond: str, label: str, idx: int, combined, noise, signal):
    stem = f"{cond}_{label}_{idx:03d}"
    # PNGs (calibrated mapping)
    Image.fromarray(to_uint8_visible(combined)).save(out_dir/cond/label/"combined"/f"{stem}_combined.png")
    Image.fromarray(to_uint8_visible(noise)).save(out_dir/cond/label/"noise"/f"{stem}_noise.png")
    if SAVE_SIGNAL_PNG:
        Image.fromarray(to_uint8_visible_signal(signal)).save(out_dir/cond/label/"signal"/f"{stem}_signal.png")
    # Optional .npy floats
    if SAVE_NPY:
        np.save(out_dir/cond/label/"combined"/f"{stem}_combined.npy", combined)
        np.save(out_dir/cond/label/"noise"/f"{stem}_noise.npy", noise)
        np.save(out_dir/cond/label/"signal"/f"{stem}_signal.npy", signal)

def export_dataset():
    OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
    conditions = {
        "equal":  {"contrast": PEAK_CONTRAST_EQUAL,  "target_snr": TARGET_SNRS["equal"]},
        "weak":   {"contrast": PEAK_CONTRAST_WEAK,   "target_snr": TARGET_SNRS["weak"]},
        "strong": {"contrast": PEAK_CONTRAST_STRONG, "target_snr": TARGET_SNRS["strong"]},
    }
    # Manifest
    (OUTPUT_DIR/"manifest.json").write_text(json.dumps({
        "MEAN_LUMINANCE": MEAN_LUMINANCE, "NOISE_SD": NOISE_SD, "FIELD_DEG": FIELD_DEG,
        "PIXELS_PER_DEG": PIXELS_PER_DEG, "IMG_SIZE": IMG_SIZE, "SIGMA_DEG": SIGMA_DEG,
        "SIGMA_PX": SIGMA_PX, "DISPLAY_GAIN": DISPLAY_GAIN,
        "PEAK_CONTRAST_EQUAL": PEAK_CONTRAST_EQUAL,
        "PEAK_CONTRAST_WEAK": PEAK_CONTRAST_WEAK,
        "PEAK_CONTRAST_STRONG": PEAK_CONTRAST_STRONG,
        "TARGET_SNRS": TARGET_SNRS, "TRIALS_PER_CONDITION": TRIALS_PER_CONDITION
    }, indent=2))

    # CSV metadata
    meta_path = OUTPUT_DIR/"metadata.csv"
    with meta_path.open("w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["trial_id","condition","label","contrast","target_snr",
                         "mean_noise","sd_noise","rms_contrast","empirical_snr_template",
                         "combined_png","noise_png","signal_png",
                         "combined_npy","noise_npy","signal_npy"])

        for cond, info in conditions.items():
            ensure_dirs(OUTPUT_DIR, cond)
            contrast = info["contrast"]; target_snr = info["target_snr"]
            # template norm on this grid (unit peak)
            tpl = gaussian_blob(IMG_SIZE, SIGMA_PX, amplitude=1.0)
            K = math.sqrt(np.sum(tpl**2))
            emp_snr_template = (contrast * MEAN_LUMINANCE * K) / NOISE_SD

            for i in range(1, TRIALS_PER_CONDITION+1):
                # Absent
                comb, noi, sig = make_stimulus(present=False, peak_contrast=contrast)
                save_trial(OUTPUT_DIR, cond, "absent", i, comb, noi, sig)
                mean_n, sd_n = float(np.mean(noi)), float(np.std(noi, ddof=1))
                rms = sd_n / mean_n
                writer.writerow([i,cond,"absent",contrast,target_snr,mean_n,sd_n,rms,emp_snr_template,
                                 str(OUTPUT_DIR/cond/"absent"/"combined"/f"{cond}_absent_{i:03d}_combined.png"),
                                 str(OUTPUT_DIR/cond/"absent"/"noise"/f"{cond}_absent_{i:03d}_noise.png"),
                                 str(OUTPUT_DIR/cond/"absent"/"signal"/f"{cond}_absent_{i:03d}_signal.png") if SAVE_SIGNAL_PNG else "",
                                 str(OUTPUT_DIR/cond/"absent"/"combined"/f"{cond}_absent_{i:03d}_combined.npy") if SAVE_NPY else "",
                                 str(OUTPUT_DIR/cond/"absent"/"noise"/f"{cond}_absent_{i:03d}_noise.npy") if SAVE_NPY else "",
                                 str(OUTPUT_DIR/cond/"absent"/"signal"/f"{cond}_absent_{i:03d}_signal.npy") if SAVE_NPY else ""])

                # Present
                comb, noi, sig = make_stimulus(present=True, peak_contrast=contrast)
                save_trial(OUTPUT_DIR, cond, "present", i, comb, noi, sig)
                mean_n, sd_n = float(np.mean(noi)), float(np.std(noi, ddof=1))
                rms = sd_n / mean_n
                writer.writerow([i,cond,"present",contrast,target_snr,mean_n,sd_n,rms,emp_snr_template,
                                 str(OUTPUT_DIR/cond/"present"/"combined"/f"{cond}_present_{i:03d}_combined.png"),
                                 str(OUTPUT_DIR/cond/"present"/"noise"/f"{cond}_present_{i:03d}_noise.png"),
                                 str(OUTPUT_DIR/cond/"present"/"signal"/f"{cond}_present_{i:03d}_signal.png") if SAVE_SIGNAL_PNG else "",
                                 str(OUTPUT_DIR/cond/"present"/"combined"/f"{cond}_present_{i:03d}_combined.npy") if SAVE_NPY else "",
                                 str(OUTPUT_DIR/cond/"present"/"noise"/f"{cond}_present_{i:03d}_noise.npy") if SAVE_NPY else "",
                                 str(OUTPUT_DIR/cond/"present"/"signal"/f"{cond}_present_{i:03d}_signal.npy") if SAVE_NPY else ""])

    print(f"[OK] Dataset exported to: {OUTPUT_DIR.resolve()} with calibrated PNGs.")


## Dataset verification (load from disk and compare to targets)

In [ ]:

def empirical_snr_from_blob(blob, noise_sigma=NOISE_SD):
    return math.sqrt(float(np.sum(blob**2))) / noise_sigma

def verify_dataset(root="perceptual_dataset_calibrated", sample=20, use_npy=True, tol=0.05):
    root = Path(root)
    meta = root/"metadata.csv"
    if not meta.exists():
        raise FileNotFoundError(f"No metadata.csv under {root}")

    rows = []
    with meta.open("r", newline="") as f:
        reader = csv.DictReader(f)
        for r in reader:
            rows.append(r)

    # group by (condition,label)
    groups = {}
    for r in rows:
        key = (r["condition"], r["label"])
        groups.setdefault(key, []).append(r)

    # template norm
    tpl = gaussian_blob(IMG_SIZE, SIGMA_PX, amplitude=1.0)
    K = math.sqrt(np.sum(tpl**2))

    def load_arr(png_path, npy_path):
        p_npy = Path(npy_path)
        if use_npy and npy_path and p_npy.exists():
            return np.load(p_npy)
        # Fallback to PNG (already calibrated to 8-bit; convert back to float in counts)
        arr = np.array(Image.open(png_path)).astype(float) / DISPLAY_GAIN
        return arr

    all_ok = True
    for (cond, label), items in groups.items():
        n = min(sample, len(items))
        subset = random.sample(items, n)
        means, sds, rmss, snrs = [], [], [], []
        contrast = float(subset[0]["contrast"])
        target_snr = float(subset[0]["target_snr"])
        template_snr = (contrast * MEAN_LUMINANCE * K) / NOISE_SD

        for r in subset:
            noise = load_arr(r["noise_png"], r["noise_npy"])
            means.append(float(noise.mean()))
            sds.append(float(noise.std(ddof=1)))
            rmss.append(float(noise.std(ddof=1) / noise.mean()))
            if label == "present":
                signal = load_arr(r["signal_png"], r["signal_npy"])
                snrs.append(empirical_snr_from_blob(signal, NOISE_SD))

        def within(x, t):
            return abs(x - t) / max(abs(t), 1e-9) <= tol

        m_mean = float(np.mean(means))
        m_sd   = float(np.mean(sds))
        m_rms  = float(np.mean(rmss))
        m_snr  = float(np.mean(snrs)) if snrs else float("nan")

        ok_mean = within(m_mean, MEAN_LUMINANCE)
        ok_sd   = within(m_sd, NOISE_SD)
        ok_rms  = within(m_rms, NOISE_SD/MEAN_LUMINANCE)
        ok_snr  = True if not snrs else within(m_snr, target_snr)

        print(f"\n[Group] {cond}/{label}  n={n}")
        print(f"  mean luminance: {m_mean:.3f} (target {MEAN_LUMINANCE:.3f}) {'OK' if ok_mean else '!!'}")
        print(f"  noise sd     : {m_sd:.3f} (target {NOISE_SD:.3f}) {'OK' if ok_sd else '!!'}")
        print(f"  rms contrast : {m_rms:.4f} (target {NOISE_SD/MEAN_LUMINANCE:.4f}) {'OK' if ok_rms else '!!'}")
        if snrs:
            print(f"  empirical SNR: {m_snr:.2f} (target {target_snr:.2f}; template {template_snr:.2f}) {'OK' if ok_snr else '!!'}")

        all_ok &= ok_mean and ok_sd and ok_rms and ok_snr

    print("\n[RESULT] " + ("PASS ✔️ within tolerance" if all_ok else "FAIL ❌ exceeded tolerance"))
    return all_ok


## Gallery: each image type for all conditions (calibrated)

In [ ]:

conditions = {
    "equal":  {"contrast": PEAK_CONTRAST_EQUAL},
    "weak":   {"contrast": PEAK_CONTRAST_WEAK},
    "strong": {"contrast": PEAK_CONTRAST_STRONG},
}

fig, axes = plt.subplots(len(conditions), 6, figsize=(18, 3*len(conditions)))

for row, (cond, info) in enumerate(conditions.items()):
    contrast = info["contrast"]
    # absent / present
    c_abs, n_abs, s_abs = make_stimulus(present=False, peak_contrast=contrast)
    c_pre, n_pre, s_pre = make_stimulus(present=True,  peak_contrast=contrast)

    # Absent
    axes[row, 0].imshow(to_uint8_visible(c_abs), cmap="gray", vmin=0, vmax=255)
    axes[row, 0].set_title(f"{cond.capitalize()} - Absent Combined"); axes[row, 0].axis("off")
    axes[row, 1].imshow(to_uint8_visible(n_abs), cmap="gray", vmin=0, vmax=255)
    axes[row, 1].set_title(f"{cond.capitalize()} - Absent Noise"); axes[row, 1].axis("off")
    axes[row, 2].imshow(to_uint8_visible_signal(s_abs), cmap="gray", vmin=0, vmax=255)
    axes[row, 2].set_title(f"{cond.capitalize()} - Absent Signal"); axes[row, 2].axis("off")

    # Present
    axes[row, 3].imshow(to_uint8_visible(c_pre), cmap="gray", vmin=0, vmax=255)
    axes[row, 3].set_title(f"{cond.capitalize()} - Present Combined"); axes[row, 3].axis("off")
    axes[row, 4].imshow(to_uint8_visible(n_pre), cmap="gray", vmin=0, vmax=255)
    axes[row, 4].set_title(f"{cond.capitalize()} - Present Noise"); axes[row, 4].axis("off")
    axes[row, 5].imshow(to_uint8_visible_signal(s_pre), cmap="gray", vmin=0, vmax=255)
    axes[row, 5].set_title(f"{cond.capitalize()} - Present Signal"); axes[row, 5].axis("off")

plt.tight_layout(); plt.show()
